## Demonstrate **hklpy2**'s API

A working notebook as the package is being developed.
***Anything*** could change.

In [1]:
import datetime
import hklpy2
print(f"{datetime.datetime.now()}")
print(f"{hklpy2.__version__=}")
print(f"{hklpy2.solvers()=}")

2024-05-22 00:23:44.218915
hklpy2.__version__='0.0.12.dev0+g97da355.d20240522'
hklpy2.solvers()={'hkl_soleil': 'hklpy2.backends.hkl_soleil:HklSolver', 'no_op': 'hklpy2.backends.no_op:NoOpSolver', 'th_tth': 'hklpy2.backends.th_tth_q:ThTthSolver'}


In [2]:
from ophyd import Component as Cpt
from ophyd import Kind
from ophyd import PseudoSingle
from ophyd import SoftPositioner

NORMAL_HINTED = Kind.hinted | Kind.normal

class Fourc(hklpy2.DiffractometerBase):
    """Test case."""

    h = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    k = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    l = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741

    theta = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind=NORMAL_HINTED)
    chi = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind=NORMAL_HINTED)
    phi = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind=NORMAL_HINTED)
    ttheta = Cpt(SoftPositioner, limits=(-170, 170), init_pos=0, kind=NORMAL_HINTED)

    h2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    k2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    l2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741

    psi = Cpt(SoftPositioner, limits=(-170, 170), init_pos=0, kind=NORMAL_HINTED)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.set_solver("hkl_soleil", "E4CV", engine="hkl")
        self._solver.auto_assign_axes(self)

fourc = Fourc("", name="fourc")
print(f"{fourc._solver=}")
print(f"{fourc._solver.pseudo_axis_names=}")
print(f"{fourc._solver.real_axis_names=}")
print(f"{fourc._solver.extra_axis_names=}")
print(f"{[axis.name for axis in fourc._solver.user_pseudos]=}")
print(f"{[axis.name for axis in fourc._solver.user_reals]=}")
print(f"{[axis.name for axis in fourc._solver.user_extras]=}")

fourc._solver=HklSolver(name='hkl_soleil', version='v5.0.0.3434', geometry='E4CV', engine='hkl', mode='bissector')
fourc._solver.pseudo_axis_names=['h', 'k', 'l']
fourc._solver.real_axis_names=['omega', 'chi', 'phi', 'tth']
fourc._solver.extra_axis_names=[]
[axis.name for axis in fourc._solver.user_pseudos]=['fourc_h', 'fourc_k', 'fourc_l']
[axis.name for axis in fourc._solver.user_reals]=['fourc_theta', 'fourc_chi', 'fourc_phi', 'fourc_ttheta']
[axis.name for axis in fourc._solver.user_extras]=['fourc_h2', 'fourc_k2', 'fourc_l2', 'fourc_psi']


In [3]:
fourc.report

{'position': FourcPseudoPos(h=0, k=0, l=0, h2=0, k2=0, l2=0)}

In [4]:
fourc.summary()

data keys (* hints)
-------------------
*fourc_chi
 fourc_geometry
*fourc_h
*fourc_h2
 fourc_h2_setpoint
 fourc_h_setpoint
*fourc_k
*fourc_k2
 fourc_k2_setpoint
 fourc_k_setpoint
*fourc_l
*fourc_l2
 fourc_l2_setpoint
 fourc_l_setpoint
*fourc_phi
*fourc_psi
 fourc_solver
*fourc_theta
*fourc_ttheta
 fourc_wavelength
 fourc_wavelength_units

read attrs
----------
solver               AttributeSignal     ('fourc_solver')
geometry             AttributeSignal     ('fourc_geometry')
wavelength           AttributeSignal     ('fourc_wavelength')
wavelength_units     AttributeSignal     ('fourc_wavelength_units')
h                    PseudoSingle        ('fourc_h')
h.readback           AttributeSignal     ('fourc_h')
h.setpoint           AttributeSignal     ('fourc_h_setpoint')
k                    PseudoSingle        ('fourc_k')
k.readback           AttributeSignal     ('fourc_k')
k.setpoint           AttributeSignal     ('fourc_k_setpoint')
l                    PseudoSingle        ('fourc_l')


In [5]:
fourc._solver.geometries()

['E4CH',
 'E4CV',
 'E6C',
 'ESRF ID01 PSIC',
 'K4CV',
 'K6C',
 'PETRA3 P09 EH2',
 'PETRA3 P23 4C',
 'PETRA3 P23 6C',
 'SOLEIL MARS',
 'SOLEIL NANOSCOPIUM ROBOT',
 'SOLEIL SIRIUS KAPPA',
 'SOLEIL SIRIUS TURRET',
 'SOLEIL SIXS MED1+2',
 'SOLEIL SIXS MED2+2',
 'SOLEIL SIXS MED2+3',
 'SOLEIL SIXS MED2+3 v2',
 'TwoC',
 'ZAXIS']

In [6]:
import math
vibranium_lattice = hklpy2.Lattice(2*math.pi)
print(f"{vibranium_lattice=}")

vibranium_lattice=Lattice(a=6.2832, b=6.2832, c=6.2832, alpha=90.0, beta=90.0, gamma=90.0)


In [7]:
vibranium = hklpy2.Sample(fourc._solver, vibranium_lattice, name="vibranium")
vibranium_lattice.digits = 2
print(f"{vibranium=}")

vibranium=Sample(name='vibranium', lattice=Lattice(a=6.28, b=6.28, c=6.28, alpha=90.0, beta=90.0, gamma=90.0))


In [8]:
vibranium.reflections.ordering = []

In [9]:
vibranium.reflections.add(
    hklpy2.Reflection(
        fourc._solver, 
        dict(h=1, k=0, l=0), 
        dict(omega=10, chi=0, phi=0, tth=20), 
        wavelength=1.00, 
        name="r1"
    )
)
vibranium.reflections.add(
    hklpy2.Reflection(
        fourc._solver, 
        dict(h=0, k=1, l=0), 
        dict(omega=10, chi=-90, phi=0, tth=20), 
        wavelength=1.00, 
        name="r2"
    )
)
for r in vibranium.reflections.ordering:
    print(f"{vibranium.reflections[r]}")

Reflection(name='r1', geometry='E4CV', pseudos={'h': 1, 'k': 0, 'l': 0}, angles={'omega': 10, 'chi': 0, 'phi': 0, 'tth': 20}, wavelength=1.0)
Reflection(name='r2', geometry='E4CV', pseudos={'h': 0, 'k': 1, 'l': 0}, angles={'omega': 10, 'chi': -90, 'phi': 0, 'tth': 20}, wavelength=1.0)


In [10]:
vibranium.reflections.swap()
print(f"{vibranium.reflections.ordering=}")
for r in vibranium.reflections.ordering:
    print(f"{vibranium.reflections[r]}")

vibranium.reflections.ordering=['r2', 'r1']
Reflection(name='r2', geometry='E4CV', pseudos={'h': 0, 'k': 1, 'l': 0}, angles={'omega': 10, 'chi': -90, 'phi': 0, 'tth': 20}, wavelength=1.0)
Reflection(name='r1', geometry='E4CV', pseudos={'h': 1, 'k': 0, 'l': 0}, angles={'omega': 10, 'chi': 0, 'phi': 0, 'tth': 20}, wavelength=1.0)
